In [83]:
# %load backtester.py
#!/usr/bin/env python3

import logging
import pandas as pd

from argparser import TradingData, convert_args_to_trading_variables, get_parsed_args
from binance_api_downloader import get_data
from simulator import run_simulation


def main():
    logging_setup()
    args = {'end_date': '2022-04-04',
 'interval': '1d',
 'pairs': ['BTCUSDT', 'ETHUSDT'],
 'start_date': '2000-01-01'}

    trading_vars = convert_args_to_trading_variables(args)

    pair_list = []
    for pair in trading_vars.pairs:
        args = {
            "ticker": pair,
            "start_date": trading_vars.start_date,
            "end_date": trading_vars.end_date,
            "interval": args["interval"],
        }

        data = get_data(args)

        if (args['interval'] == '1d'):
            # normalize the datetime to midnight if interval is 1 day
            data.index = data.index.map((lambda x: pd.Timestamp(x).replace(hour=0, minute=0, second=0)))

        multinindex = pd.MultiIndex.from_product([[pair], data.index.values], names=["pair", "open_time"])
        df = pd.DataFrame(data.values, columns=['open', 'high', 'low', 'close', 'volume'], index=multinindex)
        pair_list.append(df)

    data = pd.concat(pair_list)
    return data


def logging_setup():
    logging.basicConfig(
        level=logging.INFO,
        format="[%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        handlers=[logging.FileHandler("lastrun.log", mode="w"), logging.StreamHandler()],
    )

data = main()
data
price = data['close']
(price).vbt.plot()


import vectorbt as vbt
import plotly.express as px
import plotly.graph_objects as go

df = data

df.index.values
df.reset_index()



df = df.unstack(0)
date = pd.to_datetime('2022-03-03')
df.loc[date]

[INFO] opening existing csv file: ../data/binance/BTCUSDT:2000-01-01:2022-04-04:1d.csv
[INFO] opening existing csv file: ../data/binance/ETHUSDT:2000-01-01:2022-04-04:1d.csv


        pair   
open    BTCUSDT     43892.99000
        ETHUSDT      2947.31000
high    BTCUSDT     44101.12000
        ETHUSDT      2970.97000
low     BTCUSDT     41832.28000
        ETHUSDT      2785.00000
close   BTCUSDT     42454.00000
        ETHUSDT      2833.99000
volume  BTCUSDT     50940.61021
        ETHUSDT    446192.23530
Name: 2022-03-03 00:00:00, dtype: float64

In [17]:
data.keys()

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')

open     high      low    close         volume
pair    open_time                                                    
BTCUSDT 2017-08-17  4261.48  4485.39  4200.74  4285.08     795.150377
        2017-08-18  4285.08  4371.52  3938.77  4108.37    1199.888264
        2017-08-19  4108.37  4184.69  3850.00  4139.98     381.309763
        2017-08-20  4120.98  4211.08  4032.62  4086.29     467.083022
        2017-08-21  4069.13  4119.62  3911.79  4016.00     691.743060
...                     ...      ...      ...      ...            ...
ETHUSDT 2022-03-30  3400.51  3446.90  3331.84  3385.79  356082.523100
        2022-03-31  3385.80  3444.83  3261.35  3281.51  455091.239500
        2022-04-01  3281.51  3481.29  3210.68  3455.21  532169.239200
        2022-04-02  3455.20  3532.20  3433.14  3443.77  499893.290000
        2022-04-03  3443.77  3580.34  3412.11  3521.91  381581.768300

[3382 rows x 5 columns]